### RLE encoding and decoding

In [ ]:
from itertools import groupby


In [ ]:

def rle_encode(label):
    """
    label: 1-d numpy array
    """
    rle = [f"{k} {sum(1 for i in g)}" for k,g in groupby(label)]
    rle = " ".join(rle)
    return rle 

def rle_decode(rle):
    """
    rle: rle encoded string
    """
    rle_arr = rle.split()
    rle2label = ''.join([(rle_arr[i]+" ")*int(rle_arr[i+1]) for i in range(0,len(rle_arr)-1,2)])
    return rle2label.strip()

### Read .label and .bin file

In [1]:
import numpy as np

In [ ]:
data = np.fromfile("<.label_file_pth>",dtype=np.int32)
label =data & 0xFFFF

In [ ]:
lidar = np.fromfile("<.bin_file_pth>", dtype=np.float32)
lidar = lidar.reshape(-1, 4)

#### Judge Submission

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score

def rle_decode(rle):
    rle_arr = rle.split()
    rle2label = ''.join([(rle_arr[i]+" ")*int(rle_arr[i+1]) for i in range(0,len(rle_arr)-1,2)])
    return rle2label.strip()

def rle2vec(sub,gt):
    sub_col =  sub.columns[1]
    gt_col =  gt.columns[1]
    sub[sub_col] = sub[sub_col].apply(lambda x: rle_decode(x))
    gt[gt_col] = gt[gt_col].apply(lambda x: rle_decode(x))  
    return sub,gt



def calculate_iou(df):
    df["pcd_iou"] = df.apply(lambda x: jaccard(x.iloc[1], x.iloc[2]),axis=1)
    return df['pcd_iou'].mean()
 
def jaccard(gt, pred):
    y_true = np.array([int(i) for i in gt.split(" ")])
    y_pred = np.array([int(i) for i in pred.split(" ")])
    iou = jaccard_score(y_true, y_pred)
    return iou


In [8]:
def check_submission(pred_path,gt_path):
    sub = pd.read_csv(pred_path)
    gt = pd.read_csv(gt_path)
    sub,gt = rle2vec(sub,gt)
    
    sub.columns = list(gt.columns)
    missing_ids =set(gt.iloc[:,0].values)- set(sub.iloc[:,0].values)   
    if len(missing_ids)!=0:
        return "missing_ids: "+" ".join(list(missing_ids))
    
    merge = gt.merge(sub,on=[gt.columns[0]])

    columns =merge.columns
    merge["valid"] = merge.apply(lambda x: len(x[columns[1]])==len(x[columns[2]]),axis=1)
    
    if len(merge[merge.valid==False])!=0:
        return "Number of points is not correct in: "+" ".join(merge[merge.valid==False].iloc[:,0])
    print("getting miou")
    miou = calculate_iou(merge)
    print("miou",miou)
    team= pred_path.split("/")[1]
    return f"Test iou: {miou}"

In [9]:
pred_path = "baseline/pred_baseline.csv" # path to your prediction csv
gt_path = "gt_rle.csv"
check_submission(pred_path,gt_path)

         ID                                              Label
0  B_000000  0 11163 1 2 0 1 1 1 0 2 1 13 0 4 1 43 0 7 1 1 ...
1  B_000001  0 11177 1 4 0 1 1 1 0 5 1 3 0 1386 1 1 0 340 1...          ID                                              Label
0  T_001864  0 26848 1 41 0 2075 1 45 0 2035 1 48 0 2020 1 ...
1  T_002030  0 20665 1 50 0 2028 1 45 0 2031 1 58 0 2022 1 ...
------
         ID                                              Label
0  B_000000  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1  B_000001  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...          ID                                              Label
0  T_001864  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1  T_002030  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
------
         ID                                            Label_x  \
0  T_001864  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...   
1  T_002030  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...   

                               

'Test iou: 0.929160555091052'